# Apriori-Algorithmus

Der Apriori-Algorithmus nutzt die Monotonie-Eigenschaft, um die Anzahl der Kandidaten für häufige Itemsets frühzeitig zu verringern. In diesem Abschnitt soll sowohl die Monotonie-Eigenschaft wie auch der Apriori-Algorithmus am Beispiel gezeigt werden.

In [ ]:
from tui_dsmt.fpm import Itemset, receipts, numbers

## Inhaltsverzeichnis
- [Monotonie-Eigenschaft](#Monotonie-Eigenschaft)
- [Bestimmung der häufigen Itemsets](#Bestimmung-der-häufigen-Itemsets)
- [Einfluss des minimalen Supports](#Einfluss-des-minimalen-Supports)

## Monotonie-Eigenschaft
Betrachten Sie zuerst noch einmal die Transaktionen, die wir durch Einkäufe aufgezeichnet haben.

In [ ]:
receipts

Die Monotonie-Eigenschaft besagt nun, dass jede Teilmenge eines häufig auftretenden Itemsets selbst häufig sein muss. Kommt also die Kombination `{Brot, Milch}` in $40\%$ aller Transaktionen vor, so kommen die Teilmengen `{Brot}` und `{Milch}` ebenfalls in **mindestens** $40\%$ aller Transaktionen vor.

In [ ]:
Itemset('Brot', 'Milch').count_in(receipts)

In [ ]:
Itemset('Milch').count_in(receipts)

Die Häufigkeit der Obermenge stellt dabei eine untere Schranke dar. Die kleineren Mengen können einzeln selbstverständlich häufiger vorkommen.

In [ ]:
Itemset('Brot').count_in(receipts)

Anwenden lässt sich das Prinzip nicht nur auf einelementige Teilmengen zweielementiger Mengen. Stattdessen kann es für beliebig große Itemsets verwendet werden.

Im Folgenden verwenden wir jedoch nicht das Monotonie-Prinzip, sondern dessen Umkehrung: Wenn eine Menge nicht häufig ist, dann ist auch deren Obermenge nicht häufig. Wird Brot also nur einmal gekauft, kann die Kombination aus Brot und Milch nicht zweimal verkauft worden sein.

## Bestimmung der häufigen Itemsets
Der Apriori Algorithmus versucht nun, häufige Itemsets aus kleineren häufigen Itemsets zu erstellen. Dabei werden mit Hilfe eines minimalen Support-Wertes möglichst früh Itemsets ausgeschlossen, deren Kombinationen laut der Monotonie-Eigenschaft ebenfalls nicht mehr in Betracht zu ziehen sind. Der Algorithmus wird an folgendem Beispiel dargestellt:

In [ ]:
numbers

Außerordentlich relevant für den Algorithmus ist die Wahl des minimalen Supports. Wir setzten den Wert zunächst auf $0.5$ und kommen am Ende noch einmal auf die Wahl dieses Wertes zurück.

In [ ]:
support_threshold = 0.5

### Bestimmung der einelementigen Teilmengen
Bevor wir den Algorithmus betrachten, benötigen wir allerdings noch andere Funktionen. Beginnen wir mit dem Suchen aller einelementigen Itemsets, die den minimalen Support überschreiten.

In [ ]:
def itemsets1(transactions, min_supp):
    # leere Abbildung anlegen
    C1 = {}

    # alle Transaktionen durchgehen
    for _, items in transactions:
        # alle Elemente der Transaktion betrachten
        for element in items:
            # Element zur Abbildung hinzufügen
            # bzw. Anzahl um 1 erhöhen
            if element not in C1:
                C1[element] = 1
            else:
                C1[element] += 1

    # Elemente nach minimalem Support filtern
    # und in Itemset umwandeln
    return set(Itemset(item) for item, count in C1.items()
               if count / len(transactions) >= min_supp)


L1 = itemsets1(numbers, support_threshold)
L1

Die Menge $L_k$ enthält immer die Menge aller häufig vorkommenden Itemsets - deren Support überschreitet den minimalen Support - der Länge $k$. In der vorangegangenen Zelle wurde dementsprechend $L_1$ bestimmt.

### Generierung größerer Kandidatenmengen
Die Menge $C_k$ wiederum bezeichnet im Folgenden alle in Frage kommenden Kandidaten-Itemsets der Länge $k$. Da dem Monotonie-Kriterium folgend nur die Itemsets der Länge $k$ betrachtet werden müssen, deren Teilmengen ebenfalls häufig sind, wird $C_k$ aus $L_{k-1}$ generiert. Itemsets der Länge $k-1$, die nicht in $L_{k-1}$ enthalten sind, kommen dagegen nicht häufig genug vor und wurden bereits zuvor ausgeschlossen. Sie erneut in Betracht zu ziehen hätte also keine Vorteile.

Um $C_k$ aus $L_{k-1}$ zu generieren sind zwei Schritte notwendig:

Während des **Joins** werden die $k-1$-elementigen Itemsets aus der Menge $L_{k-1}$ zu $k$-elementigen Itemsets zusammengesetzt, sofern sie in den ersten $k-2$ Items übereinstimmen.

Das nachfolgende Beispiel zeigt die Generierung eines $4$-Itemsets aus zwei $3$-Itemsets, welche die zuvor genannte Bedingung erfüllen.

```
e1: (Brot, Eier, Käse)
                  ↓
e2: (Brot, Eier,       Reis)
                  ↓     ↓
c:  (Brot, Eier, Käse, Reis)
```

Während des **Pruning** werden alle $k-1$-elementigen Teilmengen der entstandenen $k$-elementigen Itemsets betrachtet. Wenn auch nur eine dieser Teilmengen nicht in $L_{k-1}$ enthalten ist, kann das zugehörige $k$-elementige Itemset ebenfalls nicht häufig sein und kann ohne weiteren Aufwand direkt wieder verworfen werden.

Der Algorithmus zum Generieren gestaltet sich zusammengesetzt wie folgt:

In [ ]:
def generiere_kandidaten(L, k):
    # Join
    all_C_k = set()

    for e1 in L:
        for e2 in L:
            if e1.matches(e2, k-2):
                all_C_k.add(e1.union(e2))

    # Pruning
    C_k = set()

    for c in all_C_k:
        for subset in c.subsets(k-1):
            if subset not in L:
                break
        else:
            C_k.add(c)

    return C_k


C2 = generiere_kandidaten(L1, 2)
C2

**Verständnisfrage:** Warum genügt es, Teilmengen zu kombinieren, die in den ersten $k-2$ Elementen übereinstimmen?

### Filtern nach minimalem Support
Im Vergleich zu allen möglichen Teilmengen mit $k$ Elementen ist $C_k$ bereits relativ klein. Damit $C_{k+1}$ nicht aufgebläht wird, muss $C_k$ noch zu $L_k$ reduziert werden. Dafür wird der Support aller in $C_k$ enthaltenen, $k$-elementigen Teilmengen mit dem vorgegebenen, minimalen Support verglichen und entsprechend gefiltert.

In [ ]:
def filtere_kandidaten(C_k, transactions, min_supp):
    return set(c for c in C_k if c.support_in(transactions) >= min_supp)


L2 = filtere_kandidaten(C2, numbers, support_threshold)
L2

### Finaler Algorithmus
Der Algorithmus setzt die entworfenen Bausteine nun zusammen, um Schritt für Schritt größere Itemsets zu generieren, deren Support das gegebene Minimum überschreitet. Durch die Verwendung des Monotonie-Kriteriums sowie das Filtern in jedem Schritt wird unterbunden, dass die Anzahl der in Frage kommenden Itemsets zu groß wird.

Der Algorithmus arbeitet dann wie folgt:
1. Zuerst wird die Menge $L_1$ bestimmt, die alle Itemsets der Länge $1$ enthält, die den minimalen Support-Wert überschreiten.
2. Solange die bereinigte Menge $L_{k-1}$ des vorhergehenden Durchlaufs nicht leer ist,
    1. wird die Kandidatenmenge $C_k$ aus $L_{k-1}$ berechnet und
    2. zu $L_k$ reduziert.
3. Die Vereinigung aller erzeugten Mengen häufig vorkommender Itemsets $\bigcup_i L_i$ ist das Ergebnis.

In [ ]:
def apriori(transactions, min_supp):
    # Liste der L_k anlegen
    # mit leerer Menge als Padding
    Ls = [set(), itemsets1(transactions, min_supp)]

    # k initialisieren
    k = 2

    # Schleife
    while len(Ls[-1]) > 0:
        # Kandidatengenerierung und Filter
        C_k = generiere_kandidaten(Ls[-1], k)
        L_k = filtere_kandidaten(C_k, transactions, min_supp)

        # L_k ablegen
        Ls.append(L_k)

        # k zählen
        k += 1

    # Vereinigung bilden
    result = set()
    for L in Ls:
        result.update(L)

    return result


apriori(numbers, support_threshold)

## Einfluss des minimalen Supports
Je höher der minimale Support ist, desto mehr Teilmengen werden im Laufe der Funktion `filtere_kandidaten` entfernt. Diese stehen dann natürlich auch nicht mehr im nächsten Schritt zu Verfügung, sodass erneut weniger Kandidaten generiert werden. Für die Laufzeit des Algorithmus wäre ein minimaler Support von $1$ also optimal. Gleichzeitig werden durch einen zu hohen Support allerdings Itemsets entfernt, die potentiell interessant sind.

Der Wert für den minimalen Support sollte also so niedrig wie nötig und so hoch wie möglich gewählt werden.